In [1592]:
import numpy as np

In [1713]:
class Person:
    def __init__(self, name):
        self.name = name
        self.edges = list()
        self.santa = None
        self.child = None
        self.parent = None
        self.banlist = []
    
    def __repr__(self):
        return str(self.name)
    
    def pop(self):
        x = self.santa
        self.santa = None
        return x 

    def ban(self, to_ban):
        self.banlist.append(to_ban)

In [1714]:
names = ['Alice', 'Cathy', 'Ching', 'Giacomo', 'Henry', 'Jason', 'Jeff', 'Jero', 'Tiff', 'Tim', 'Sunny', 'Liv', 'Bobo', 'Lawrence']

In [1728]:
# instantiate Person objects
p = {i: Person(i) for i in names}

In [1739]:
# Populate edges 
for i in p.values():
    for j in p.values():
        if i != j:
            i.edges.append(j)

In [1740]:
def negative(n1, *names):
    p1 = p[n1]
    for n in names:
        if p[n] in p1.edges:
            p1.edges.remove(p[n])
        if p1 in p[n].edges:
            p[n].edges.remove(p1)

In [1741]:
negative('Sunny', 'Tim', 'Giacomo', 'Henry', 'Jason', 'Jeff', 'Jero', 'Tiff', 'Bobo')
negative('Jason', 'Tim', 'Giacomo', 'Alice', 'Henry', 'Tiff', 'Jeff', 'Cathy', 'Ching')
negative('Liv', 'Cathy', 'Giacomo', 'Jeff', 'Tiff', 'Sunny', 'Jason', 'Jero')
negative('Lawrence', 'Alice', 'Cathy', 'Ching', 'Giacomo', 'Jason', 'Tiff', 'Sunny', 'Jeff')
negative('Henry', 'Ching', 'Giacomo')
negative('Cathy', 'Jeff', 'Tiff')
negative("Ching", "Tiff", "Jeff")

In [1742]:
# sort by num_edges (increasing)
num_nodes = [len(i.edges) for i in p.values()]
idx = np.argsort(num_nodes)
people = np.array([i for i in p.values()])[idx]

In [1743]:
# Constraints: 
# Secret Santa of the Secret Santa of a Recipient cannot be the Recipient 
# Each person only assigned once 
# Secret Santa - Recipient must be people who know each other well aka valid edge 

def check(lst: list):
    s = set()
    r = set()
    for i in lst:
        assert i.santa.santa != i 
        assert i.santa in i.edges
        s.add(str(i.santa.name))
        r.add(str(i.name))
    assert len(r) == len(s)

def reset(lst: list):
    for i in lst:
        i.pop()

In [1744]:
# Set up node structure 
for i in range(len(people)):
    try:
        people[i].child = people[i+1]
        people[i+1].parent = people[i]
    except IndexError:
        people[i].child = None

In [1745]:
num = []

def generate(safeguard=False):
    while True:
        iters = 0
        santa = [] 
        reset(people)
        node = people[0]

        while True:
            iters += 1
            if safeguard is True:
                if iters > 50:
                    iters = 0
                    node = people[0]
                    all_clear(people)
                    santa = []
                    iters = 0
                    continue

            # Set up pool of candidates             
            edges = [i for i in node.edges if i not in santa and i not in node.banlist]

            # Handle scenarios that necessitate backtracking, backtrack twice
            if len(edges) == 1 and edges[0].santa == node or len(edges) == 0:
                for _ in range(2):
                    if node.parent is not None:
                        node = node.parent
                        if len(santa) != 0:
                            node_santa = node.pop()
                            santa.remove(node_santa)
                try:
                    node.ban(node_santa)
                except:
                    pass
                
            else:
                # Selection is valid if santa of selection != current node
                choice = np.random.choice(edges)
                if choice.santa != node:
                    node.santa = choice
#                     print(f'{node=}')
#                     print(f'{edges=}')
#                     print(f'{choice=}')
                    try:
                        # reset banlist of child
                        node.child.banlist = []
                    except AttributeError:
                        pass
                    santa.append(choice)
                    node = node.child
                    
                    if node is None:
                        # all assigned, exit loop
                        num.append(iters)
#                         print(f'{iters=}')
                        break
                else:
                    # ban illegitimate choice and reroll
                    node.ban(choice)                    

        # Check constraints
        check(people)

        # construct dictionary
        secret_santa = {i: i.santa for i in people}
        yield secret_santa

In [1751]:
s = set()
for i in range(50000):
#     print(i)
    config = next(generate())
#     for person, santa in config.items():
#         print(f'Secret Santa : {santa}')
#         print(f'Recipient : {person}')
#         print('\n')
    s.add(str(config))

print(f'{len(s)} unique combinations generated')
print(f'Average Iters : {np.mean(num)}')
print(f'Median Iters : {np.median(num)}')

37032 unique combinations generated
Average Iters : 24.05819820928587
Median Iters : 18.0


In [1752]:
for i in next(generate()):
    print('Secret Santa : ', i.santa)
    print('Recipient    :', i)
    print('\n')

Secret Santa :  Bobo
Recipient    : Jason


Secret Santa :  Ching
Recipient    : Sunny


Secret Santa :  Liv
Recipient    : Lawrence


Secret Santa :  Henry
Recipient    : Liv


Secret Santa :  Tiff
Recipient    : Jeff


Secret Santa :  Giacomo
Recipient    : Tiff


Secret Santa :  Tim
Recipient    : Cathy


Secret Santa :  Alice
Recipient    : Ching


Secret Santa :  Jero
Recipient    : Giacomo


Secret Santa :  Cathy
Recipient    : Henry


Secret Santa :  Sunny
Recipient    : Alice


Secret Santa :  Jason
Recipient    : Jero


Secret Santa :  Lawrence
Recipient    : Tim


Secret Santa :  Jeff
Recipient    : Bobo


